In [1]:
%useLatestDescriptors
%use dataframe(1.0.0-Beta3), kandy

In [2]:
import kotlinx.datetime.LocalTime
import kotlinx.datetime.format.byUnicodePattern

val dateFormatter = LocalDate.Format { byUnicodePattern("yyyy-MM-dd") }
val timeFormatter = LocalTime.Format { byUnicodePattern("HH:mm") }

val offsets = listOf(
    0   to "PaxName",
    60  to "PaxBirthDate",
    72  to "DepartDate",
    84  to "DepartTime",
    96  to "ArrivalDate",
    108 to "ArrivalTime",
    120 to "Flight",
    126 to "CodeSh",
    132 to "From",
    138 to "Dest",
    144 to "Code",
    150 to "e-Ticket",
    168 to "TravelDoc",
    180 to "Seat",
    186 to "Meal",
    192 to "TrvCls",
    198 to "Fare",
    204 to "Baggage",
    216 to "PaxAdditionalInfo",
    276 to "AgentInfo",
    336 to "EOL",
)
val codeShMapping = mapOf(
    "YES" to true,
    "NO" to false,
)

In [3]:
import kotlinx.datetime.toJavaLocalDate
import kotlinx.datetime.toJavaLocalTime

fun String.toDate(): java.time.LocalDate? {
    return try {
        LocalDate.parse(this, dateFormatter).toJavaLocalDate()
    } catch (e: Exception) {
        null
    }
}

fun String.toTime(): java.time.LocalTime? {
    return try {
        LocalTime.parse(this, timeFormatter).toJavaLocalTime()
    } catch (e: Exception) {
        null
    }
}

In [5]:
import ds.types.raw.SirenaExport
import java.util.Scanner

val file = File("../data/Sirena-export-fixed.tab")
val data = ArrayList<SirenaExport>()

file.inputStream().use {
    val scanner = Scanner(it)
    scanner.nextLine()

    while (scanner.hasNextLine()) {
        val line = scanner.nextLine()
        val cols = Array<String>(offsets.size - 1) { i ->
            line.substring(offsets[i].first, offsets[i + 1].first).trim()
        }
        data.add(SirenaExport(
            paxName = cols[0],
            paxBirthData = cols[1].toDate(),
            departureDate = cols[2].toDate()!!,
            departureTime = cols[3].toTime()!!,
            arrivalDate = cols[4].toDate()!!,
            arrivalTime = cols[5].toTime()!!,
            flight = cols[6],
            codeSh = codeShMapping[cols[7]]!!,
            from = cols[8],
            dest = cols[9],
            code = cols[10],
            eticket = cols[11],
            travelDoc = cols[12],
            seat = cols[13],
            meal = cols[14],
            trvCls = cols[15],
            fare = cols[16],
            baggage = cols[17],
            paxAdditionalInfo = cols[18],
            agentInfo = cols[19],
        ))
    }
}

In [6]:
import ds.utils.db.inserter

inserter<SirenaExport>().insert(data)